In [1]:
] activate .

  Activating project at `/media/dasc3/SAMSUNG_SSD/dasc3/devansh/nvblox-accuracy-testing/julia_ply_viz`


In [2]:
] st

Status `/media/dasc3/SAMSUNG_SSD/dasc3/devansh/nvblox-accuracy-testing/julia_ply_viz/Project.toml`
⌃ [35d6a980] ColorSchemes v3.26.0
⌅ [3da002f7] ColorTypes v0.11.5
  [150eb455] CoordinateTransformations v0.6.3
  [a6e39b05] DecompUtil v0.1.3 `https://github.com/dev10110/DecompUtil.jl#main`
⌃ [5789e2e9] FileIO v1.16.3
  [60bf3e95] GLPK v1.2.1
⌅ [5c1252a2] GeometryBasics v0.4.11
⌃ [916415d5] Images v0.26.1
  [682c06a0] JSON v0.21.4
⌃ [283c5d60] MeshCat v0.16.2
⌃ [91a5bcdd] Plots v1.40.8
⌃ [42171d58] PlyIO v1.1.2
⌅ [67491407] Polyhedra v0.7.8
  [92933f4c] ProgressMeter v1.10.2
⌃ [6099a3de] PythonCall v0.9.23
⌃ [a8468747] QHull v0.2.5
⌃ [295af30f] Revise v3.6.0
  [6038ab10] Rotations v1.7.1
  [87eb8544] SolidModeling v0.1.0
⌃ [90137ffa] StaticArrays v1.9.7
⌃ [a759f4b9] TimerOutputs v0.5.24
Info Packages marked with ⌃ and ⌅ have new versions available. Those with ⌃ may be upgradable, but those with ⌅ are restricted by compatibility constraints from upgrading. To see why use `status --outdat

In [3]:

using MeshCat, ColorTypes, GeometryBasics
using FileIO, PlyIO
using LinearAlgebra, StaticArrays
using Polyhedra
using ColorSchemes
using ProgressMeter
using DelimitedFiles
using Printf 
include("src/vis_helper.jl")
# include("src/direct_mapping.jl")
VH = VisHelper;
# DM = DirectMapping

using CoordinateTransformations, Rotations, StaticArrays




In [4]:
function read_trajectory(traj_file)
    M = readdlm(traj_file, ' ', Float64, '\n')
    return map( v -> SMatrix{4,4}(collect(reshape(v, 4,4)')), eachrow(M) )
end
function trajectory_path(base_path)
    return base_path * "/traj.txt"
end  

trajectory_path (generic function with 1 method)

In [5]:
vis = Visualizer()
open(vis)

[ Info: Listening on: 127.0.0.1:8700, thread id: 1
┌ Info: MeshCat server started. You can open the visualizer by visiting the following URL in your browser:
└ http://127.0.0.1:8700


Process(`xdg-open http://127.0.0.1:8700`, ProcessExited(0))

Opening in existing browser session.


In [6]:
env_name = "office0"
alg = "certified"
Σ = 1e-6 * I(6)

basedir = "../data/Replica/$(env_name)/"
data_root_dir = "./esdf_data/office0/"
esdf_root_dir = data_root_dir * "/esdf/"

# grab the true traj
true_traj = read_trajectory(trajectory_path(basedir))
est_traj = read_trajectory(data_root_dir * "perturbed.txt")
    

# grab the ground truth pointcloud
groundtruth_mesh = "../data/Replica/$(env_name)_mesh.ply"
gt_mesh = VH.PlyMesh(load_ply(groundtruth_mesh))



gt_mesh_center = sum(gt_mesh.coordinates) / length(gt_mesh.coordinates)



3-element Point{3, Float32} with indices SOneTo(3):
  0.30573833
 -0.7496212
  0.22674635

In [240]:

# draw the gt_mesh into the vis
# delete!(vis)

# initialize viz
setprop!(vis["/Background"], "visible", false)
setprop!(vis["/Grid"], "visible", false)
setprop!(vis["/Axes"], "visible", false)

# set transfom of viz
settransform!(vis["shifted"], Translation(-gt_mesh_center))

# draw the mesh
VH.draw_mesh!(vis["shifted"]["gt_mesh"], VH.cut_ceiling(gt_mesh, 1.3)) # assumed to be already cut ceiling

# draw the camera cone
c = Cone(Point3f(0,0,0.4), Point3f(0,0,0.0), 0.2f0)
setobject!(vis["shifted"]["camera_cone"], c, MeshPhongMaterial(;color=RGB(0.3,0.3,0.3)))



# get the correct rotation for the camera

for angle = 0:0.05:(2π)
    print(angle)

    # set the camera position
    camera_r = 3.5
    camera_z = 4.0 # + gt_mesh_center[3]
    camera_x = camera_r * cos(angle) # + gt_mesh_center[1]
    camera_y = camera_r * sin(angle) # + gt_mesh_center[2]
    T_camera = Translation(camera_x, camera_y, camera_z) # ∘ T_center_of_attention


    settransform!(vis["/Cameras/default"], T_camera)

        # force camera to be exactly where I set the transform to be
    setprop!(vis["/Cameras/default/rotated/<object>"], "position", [0,0,0])

    sleep(0.1)
end

0.00.050.10.150000000000000020.20.250.300000000000000040.350000000000000030.40.450.50.550.60000000000000010.650.70000000000000010.750.80.85000000000000010.90.95000000000000011.01.051.11.15000000000000011.20000000000000021.251.31.351.40000000000000011.45000000000000021.51.551.61.65000000000000011.70000000000000021.751.81.851.90000000000000011.95000000000000022.02.05000000000000032.12.152.22.252.30000000000000032.352.40000000000000042.452.52.55000000000000032.62.65000000000000042.72.752.80000000000000032.852.90000000000000042.953.03.05000000000000033.13.15000000000000043.23.253.30000000000000033.353.40000000000000043.453.53.55000000000000033.63.65000000000000043.73.753.80000000000000033.853.90000000000000043.954.04.054.10000000000000054.154.24.254.34.35000000000000054.44.454.54.554.60000000000000054.654.74.754.8000000000000014.85000000000000054.94.955.05.0500000000000015.10000000000000055.155.25.255.3000000000000015.35000000000000055.45.455.55.5500000000000015.60000000000000055.655.75.75

In [7]:
function esdf_filename(k; esdf_root_dir, mode="baseline")
    
    if mode == "baseline"
        fn = @sprintf "%04d_esdf.ply" k
        return esdf_root_dir * fn
    else
        fn = @sprintf "%04d_certi_esdf.ply" k
        return esdf_root_dir * fn
    end
end

function get_esdf_pointcloud(ply; threshold=0.02 * sqrt(3)/2, color_lo = RGB(1,0,0), color_hi=RGB(0,1,0))
    # create pts
    pts = map((x,y,z) -> Point3f(x,y,z), 
        ply["vertex"]["x"], 
        ply["vertex"]["y"],
        ply["vertex"]["z"])
    
    # create colors
    colors = map( i-> i >= threshold ? color_hi : color_lo, ply["vertex"]["intensity"])
    
    return PointCloud(pts, colors)
end

function get_processed_esdf_pc(esdf_path, T_map_body_est, T_inertial_body_true; 
        slice_z=-0.6, 
        thresh = 0.02 * sqrt(3)/2, 
        color_lo = RGBA(1,0,0, 0.5), 
        color_hi = RGBA(0,1,0, 0.5)
    )
    
    esdf_ply = load_ply(esdf_path)
    T =  T_inertial_body_true * inv(T_map_body_est) 
    rot_esdf_ply = VH.rotate_ply(esdf_ply, T)
    sliced_esdf = VH.slice_ply(rot_esdf_ply,  z_lims=(slice_z -0.05, slice_z + 0.05))
    
    return get_esdf_pointcloud(sliced_esdf; threshold=thresh, color_lo=color_lo, color_hi=color_hi)
end

get_processed_esdf_pc (generic function with 1 method)

In [232]:
minimum(map(x->x[3], gt_mesh.coordinates))

-1.1688691f0

In [233]:
maximum(map(x->x[3], gt_mesh.coordinates))

1.8229918f0

In [243]:


# draw the gt_mesh into the vis
delete!(vis)

# initialize viz
setprop!(vis["/Background"], "visible", false)
setprop!(vis["/Grid"], "visible", false)
setprop!(vis["/Axes"], "visible", false)

# set transfom of viz
settransform!(vis["shifted"], Translation(-gt_mesh_center))

# draw the mesh
VH.draw_mesh!(vis["shifted"]["gt_mesh"], VH.cut_ceiling(gt_mesh, 1.3)) # assumed to be already cut ceiling

# draw the camera cone
c = Cone(Point3f(0,0,0.4), Point3f(0,0,0.0), 0.2f0)
setobject!(vis["shifted"]["camera_cone"], c, MeshPhongMaterial(;color=RGB(0.3,0.3,0.3)))



N = 2000
is_first = true
last_cleared = 0
for k = 1594:50:1600
    
    # try clearing gc
    if last_cleared < k - 10
        GC.gc()
        last_cleared = k
        println("gc: $k")
    end
    
    
    # check if the esdf exists
    esdf_fn = esdf_filename(k; esdf_root_dir=esdf_root_dir, mode="baseline")

    # update esdf if it exists
    if isfile(esdf_fn)
        println("updating esdf at $k")

        # grab the transforms
        T_map_body_est = est_traj[k]
        T_inertial_body_true = true_traj[k]
        
        
        slice_z = -0.65  # + 0.3 * sin(2 * π * k / N * 4)

        # grab the esdf
        esdf_pc = get_processed_esdf_pc(esdf_fn, T_map_body_est, T_inertial_body_true, 
            slice_z=slice_z,
            thresh=0.02 * sqrt(3)/2)

        # plot
        VH.draw_pointcloud!(vis["shifted"]["pc"], esdf_pc)
    end
    
    # transform all sfcs into the correct frame
    T_odom_body_k = true_traj[k]

    # plot camera
    settransform!(vis["shifted"]["camera_cone"], VH.get_transform(T_odom_body_k))

    # plot the past points
    past_points = [T[1:3, 4] for T in true_traj[1:k]]
    VH.draw_points!(vis["shifted"]["past_poses"], past_points; color=RGB(0,0,0), size=0.005)

    
    # force camera to be exactly where I set the transform to be
    setprop!(vis["/Cameras/default/rotated/<object>"], "position", [0,0,0])

    # get the correct rotation for the camera
    num_revolutions = 2.0
    angle = k / N * num_revolutions * 2π

    # set the camera position
    camera_r = 4.5
    camera_z = 5.0
    camera_x = camera_r * cos(angle)
    camera_y = camera_r * sin(angle)
    T_camera = Translation(camera_x, camera_y, camera_z) # ∘ T_center_of_attention

    settransform!(vis["/Cameras/default"], T_camera)

    if is_first
        sleep(1)
        s = readline()
        if s == "n"
            break
        end
        is_first = false
    else
        sleep(0.1)
    end

    # try to save the image to file
    println("saving $k: ")
#     save_image(vis)
#     println("..done $k")


end


stdin> n
gc: 1594
updating esdf at 1594


In [247]:
setprop!(vis["/Cameras/default/rotated/<object>"], "position", [10,0,0])

MeshCat Visualizer with path /Cameras/default/rotated/<object> at http://127.0.0.1:8707

In [10]:
# grab the transforms
k=1594
esdf_fn = esdf_filename(k; esdf_root_dir=esdf_root_dir, mode="baseline")
T_map_body_est = est_traj[k]
T_inertial_body_true = true_traj[k]


slice_z = -0.6  # + 0.3 * sin(2 * π * k / N * 4)

# grab the esdf
esdf_pc = get_processed_esdf_pc(esdf_fn, T_map_body_est, T_inertial_body_true, 
slice_z=slice_z,
thresh=0.02 * sqrt(3)/2)

# plot
VH.draw_pointcloud!(vis["shifted"]["pc"], esdf_pc)

MeshCat Visualizer with path /meshcat/shifted/pc at http://127.0.0.1:8700

In [11]:
# draw the mesh
VH.draw_mesh!(vis["shifted"]["gt_mesh"], VH.cut_ceiling(gt_mesh, 1.3)) # assumed to be already cut ceiling


MeshCat Visualizer with path /meshcat/shifted/gt_mesh at http://127.0.0.1:8700

In [15]:
traj_pts = [T[1:3, 4] for T in true_traj]
VH.draw_points!(vis["shifted"]["true_traj"], map(p->Point3f(p...), traj_pts); size=0.02)

MeshCat Visualizer with path /meshcat/shifted/true_traj at http://127.0.0.1:8700

In [16]:
est_traj_pts = [T[1:3, 4] for T in est_traj]
VH.draw_points!(vis["shifted"]["est_traj"], est_traj_pts; size=0.02, color=RGB(0,0,0))

MeshCat Visualizer with path /meshcat/shifted/est_traj at http://127.0.0.1:8700